In [ ]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_census_income
import tensorflow.keras.backend as K
import argparse
from scalelayer import  ScaleLayer

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred


def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

models_map = {
    'a': "models/gated_models/adult_a_gated_4_0.3_0.2_p-0.3_p0.15.h5",
    'r': "models/gated_models/adult_r_gated_4_0.3_0.2_p-0.95_p0.8.h5",
    'g': "models/gated_models/adult_g_gated_4_0.3_0.2_p-0.6_p0.1.h5",
    'a&r': "models/gated_models/adult_a&r_gated_4_0.3_0.2_p-0.35_p0.25.h5",
    'a&g': "models/gated_models/adult_a&g_gated_4_0.3_0.2_p-0.3_p0.25.h5",
    'r&g': "models/gated_models/adult_r&g_gated_4_0.3_0.2_p-0.9_p0.8.h5",
}

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'layer5'), ('layer5', 'layer6')]

from preprocessing import pre_census_income
from tensorflow.keras.models import Model

for layer_index in range(5, 6):
    print("*"*10, "layer", layer_index+1)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]
        
        model_path = models_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        print(model.get_layer('layer6').get_weights())
#         model.summary()

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_census_income.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_census_income.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_census_income.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()
        
        for m, out in enumerate(inter_output_ori):
            print("index:",m, out)

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

********** layer 6
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
[array([[ 2.9423175 ],
       [-1.499268  ],
       [-0.73832774],
       [-0.76415855],
       [ 0.704021  ],
       [-4.6148725 ],
       [-0.95385844],
       [-0.68753624],
       [-1.078599  ],
       [ 1.7334886 ]], dtype=float32), array([-0.18941183], dtype=float32)]
index: 0 [0.17130086]
index: 1 [0.00017318]
index: 2 [0.08544636]
index: 3 [0.1719676]
index: 4 [0.09347486]
index: 5 [0.43749982]
index: 6 [0.06411612]
index: 7 [0.0733448]
index: 8 [0.47903845]
index: 9 [0.00839221]
index: 10 [0.5057362]
index: 11 [0.65876573]
index: 12 [0.6385702]
index: 13 [0.6036915]
index: 14 [0.71504134]
index: 15 [0.01873919]
index: 16 [0.4721572]
index: 17 [0.5

index: 581 [0.4966442]
index: 582 [0.22214234]
index: 583 [0.08745113]
index: 584 [0.59230316]
index: 585 [0.09027717]
index: 586 [0.24980268]
index: 587 [0.12258515]
index: 588 [0.35266882]
index: 589 [0.454067]
index: 590 [0.45760542]
index: 591 [0.1710585]
index: 592 [0.2184104]
index: 593 [0.01418722]
index: 594 [0.15611088]
index: 595 [0.42954847]
index: 596 [0.06544009]
index: 597 [0.6377214]
index: 598 [0.41767752]
index: 599 [0.53060913]
index: 600 [0.16405037]
index: 601 [0.00498471]
index: 602 [0.5870798]
index: 603 [0.06640846]
index: 604 [0.5297827]
index: 605 [0.7152078]
index: 606 [0.11216402]
index: 607 [0.695979]
index: 608 [0.76965207]
index: 609 [0.01792273]
index: 610 [0.07286429]
index: 611 [0.47037995]
index: 612 [0.45824602]
index: 613 [0.04575416]
index: 614 [0.6207359]
index: 615 [0.5976567]
index: 616 [0.05013016]
index: 617 [0.04742065]
index: 618 [0.15984839]
index: 619 [0.11797798]
index: 620 [0.45760542]
index: 621 [0.40507078]
index: 622 [0.14298558]
index

index: 1180 [0.6405973]
index: 1181 [0.5549453]
index: 1182 [0.1542106]
index: 1183 [0.09600884]
index: 1184 [0.16308478]
index: 1185 [0.19544476]
index: 1186 [0.21088201]
index: 1187 [0.3417501]
index: 1188 [0.03638515]
index: 1189 [0.5972597]
index: 1190 [0.34113532]
index: 1191 [0.13627028]
index: 1192 [0.00786933]
index: 1193 [0.19189581]
index: 1194 [0.08071366]
index: 1195 [0.76547545]
index: 1196 [0.16979957]
index: 1197 [0.03596422]
index: 1198 [0.25177848]
index: 1199 [0.07070038]
index: 1200 [0.44497317]
index: 1201 [0.7037542]
index: 1202 [0.36113912]
index: 1203 [0.09027717]
index: 1204 [0.16513842]
index: 1205 [0.6682435]
index: 1206 [0.34519875]
index: 1207 [0.09225792]
index: 1208 [0.06855047]
index: 1209 [0.02973562]
index: 1210 [0.13596418]
index: 1211 [0.09073064]
index: 1212 [0.22687835]
index: 1213 [0.15190402]
index: 1214 [0.45235667]
index: 1215 [0.10437876]
index: 1216 [0.0424566]
index: 1217 [0.36771125]
index: 1218 [0.09600884]
index: 1219 [0.08492169]
index: 1

index: 1760 [0.06170204]
index: 1761 [0.04426202]
index: 1762 [0.11013824]
index: 1763 [0.10009813]
index: 1764 [0.03505167]
index: 1765 [0.08994016]
index: 1766 [0.07514995]
index: 1767 [0.24364653]
index: 1768 [0.10479271]
index: 1769 [0.03429377]
index: 1770 [0.36200732]
index: 1771 [0.02239132]
index: 1772 [0.42165458]
index: 1773 [0.62578005]
index: 1774 [0.35581636]
index: 1775 [0.07292268]
index: 1776 [0.0945524]
index: 1777 [0.56052315]
index: 1778 [0.26988977]
index: 1779 [0.63228387]
index: 1780 [0.51651603]
index: 1781 [0.19687086]
index: 1782 [0.08168468]
index: 1783 [0.1739304]
index: 1784 [0.10009813]
index: 1785 [0.12432402]
index: 1786 [0.39435458]
index: 1787 [0.04251891]
index: 1788 [0.08525234]
index: 1789 [0.6835727]
index: 1790 [0.34647858]
index: 1791 [0.04995644]
index: 1792 [0.3767048]
index: 1793 [0.13596418]
index: 1794 [0.41528523]
index: 1795 [0.03677791]
index: 1796 [0.01368028]
index: 1797 [0.00726384]
index: 1798 [0.5192532]
index: 1799 [0.28765363]
index

index: 2342 [0.66464937]
index: 2343 [0.08397007]
index: 2344 [0.6551219]
index: 2345 [0.6844282]
index: 2346 [0.2586011]
index: 2347 [0.61904705]
index: 2348 [0.09144112]
index: 2349 [0.6973956]
index: 2350 [0.11873254]
index: 2351 [0.76965207]
index: 2352 [0.5693359]
index: 2353 [0.43168113]
index: 2354 [0.57923037]
index: 2355 [0.11672539]
index: 2356 [0.4594249]
index: 2357 [0.4269023]
index: 2358 [0.18703955]
index: 2359 [0.4721572]
index: 2360 [0.29398155]
index: 2361 [0.6096948]
index: 2362 [0.5192532]
index: 2363 [0.48152]
index: 2364 [0.04473159]
index: 2365 [0.44054696]
index: 2366 [0.06217942]
index: 2367 [0.11061141]
index: 2368 [0.03084505]
index: 2369 [0.04797247]
index: 2370 [0.10882026]
index: 2371 [0.20264441]
index: 2372 [0.08538917]
index: 2373 [0.02244577]
index: 2374 [0.2372751]
index: 2375 [0.27411994]
index: 2376 [0.00223383]
index: 2377 [0.45235667]
index: 2378 [0.623515]
index: 2379 [0.44143137]
index: 2380 [0.05167073]
index: 2381 [0.03666916]
index: 2382 [0.4

...

index: 28839 [0.18487343]
index: 28840 [0.02810407]
index: 28841 [0.69791067]
index: 28842 [0.73942244]
index: 28843 [0.4518664]
index: 28844 [0.16780853]
index: 28845 [0.00770155]
index: 28846 [0.3807959]
index: 28847 [0.3938818]
index: 28848 [0.37018389]
index: 28849 [0.3938818]
index: 28850 [0.20205912]
index: 28851 [0.14173847]
index: 28852 [0.05920058]
index: 28853 [0.4825363]
index: 28854 [0.07349381]
index: 28855 [0.03872091]
index: 28856 [0.01879275]
index: 28857 [0.06383818]
index: 28858 [0.10741448]
index: 28859 [0.08366117]
index: 28860 [0.19215879]
index: 28861 [0.20402852]
index: 28862 [0.75983065]
index: 28863 [0.24980268]
index: 28864 [0.01942578]
index: 28865 [0.293571]
index: 28866 [0.5549054]
index: 28867 [0.29398155]
index: 28868 [0.23706421]
index: 28869 [0.60390127]
index: 28870 [0.6573092]
index: 28871 [0.00779858]
index: 28872 [0.32499862]
index: 28873 [0.519747]
index: 28874 [0.75084627]
index: 28875 [0.06595856]
index: 28876 [0.0595271]
index: 28877 [0.56335294

index: 29387 [0.14566863]
index: 29388 [0.20799536]
index: 29389 [0.00498426]
index: 29390 [0.18476653]
index: 29391 [0.09852913]
index: 29392 [0.18755779]
index: 29393 [0.1399965]
index: 29394 [0.02893275]
index: 29395 [0.09445417]
index: 29396 [0.04512751]
index: 29397 [0.05090567]
index: 29398 [0.066163]
index: 29399 [0.62114716]
index: 29400 [0.30333632]
index: 29401 [0.20264441]
index: 29402 [0.05547816]
index: 29403 [0.10650754]
index: 29404 [0.3624588]
index: 29405 [0.10051256]
index: 29406 [0.4844176]
index: 29407 [0.1448136]
index: 29408 [0.31542292]
index: 29409 [0.11058915]
index: 29410 [0.22687835]
index: 29411 [0.09618071]
index: 29412 [0.02014422]
index: 29413 [0.06647745]
index: 29414 [0.05963796]
index: 29415 [0.09471244]
index: 29416 [0.41207182]
index: 29417 [0.08334231]
index: 29418 [0.25603756]
index: 29419 [0.25084174]
index: 29420 [0.35977894]
index: 29421 [0.5549054]
index: 29422 [0.57521373]
index: 29423 [0.13713032]
index: 29424 [0.06951603]
index: 29425 [0.052

index: 29944 [0.09600884]
index: 29945 [0.20905098]
index: 29946 [0.1455265]
index: 29947 [0.22117409]
index: 29948 [0.11545706]
index: 29949 [0.4257767]
index: 29950 [0.15790024]
index: 29951 [0.03630605]
index: 29952 [0.3144244]
index: 29953 [0.34568948]
index: 29954 [0.19187683]
index: 29955 [0.26367748]
index: 29956 [0.28154072]
index: 29957 [0.11115465]
index: 29958 [0.04726583]
index: 29959 [0.4269023]
index: 29960 [0.06110668]
index: 29961 [0.11294568]
index: 29962 [0.6879871]
index: 29963 [0.38883328]
index: 29964 [0.07462752]
index: 29965 [0.59230316]
index: 29966 [0.2402136]
index: 29967 [0.06795391]
index: 29968 [0.64803725]
index: 29969 [0.09600884]
index: 29970 [0.00199771]
index: 29971 [0.49503592]
index: 29972 [0.40981162]
index: 29973 [0.3938818]
index: 29974 [0.35937652]
index: 29975 [0.09618071]
index: 29976 [0.06170204]
index: 29977 [0.31632626]
index: 29978 [0.30302966]
index: 29979 [0.00958827]
index: 29980 [0.26945925]
index: 29981 [0.11505926]
index: 29982 [0.005

index: 30511 [0.4505644]
index: 30512 [0.19792694]
index: 30513 [0.46036783]
index: 30514 [0.18756774]
index: 30515 [0.06122905]
index: 30516 [0.0300746]
index: 30517 [0.5606924]
index: 30518 [0.3193094]
index: 30519 [0.06055439]
index: 30520 [0.37965238]
index: 30521 [0.11061141]
index: 30522 [0.3140847]
index: 30523 [0.12354577]
index: 30524 [0.07626662]
index: 30525 [0.5527238]
index: 30526 [0.6092622]
index: 30527 [0.33051765]
index: 30528 [0.15955251]
index: 30529 [0.6207359]
index: 30530 [0.33873975]
index: 30531 [0.01460147]
index: 30532 [0.17049813]
index: 30533 [0.01260346]
index: 30534 [0.29398155]
index: 30535 [0.17147163]
index: 30536 [0.1204859]
index: 30537 [0.06170532]
index: 30538 [0.293571]
index: 30539 [0.00254998]
index: 30540 [0.06087631]
index: 30541 [0.3335633]
index: 30542 [0.519747]
index: 30543 [0.04764372]
index: 30544 [0.06465092]
index: 30545 [0.17322841]
index: 30546 [0.09743544]
index: 30547 [0.0595271]
index: 30548 [0.2251657]
index: 30549 [0.05154964]
in

index: 31075 [0.3507781]
index: 31076 [0.15287507]
index: 31077 [0.09324256]
index: 31078 [0.04022768]
index: 31079 [0.4085341]
index: 31080 [0.03320643]
index: 31081 [0.09813538]
index: 31082 [0.04582241]
index: 31083 [0.15055862]
index: 31084 [0.34601003]
index: 31085 [0.06843874]
index: 31086 [0.12909481]
index: 31087 [0.06663123]
index: 31088 [0.12548158]
index: 31089 [0.1798645]
index: 31090 [0.6207359]
index: 31091 [0.26328182]
index: 31092 [0.587737]
index: 31093 [0.51719683]
index: 31094 [0.67427206]
index: 31095 [0.6910595]
index: 31096 [0.08920401]
index: 31097 [0.00739393]
index: 31098 [0.3938818]
index: 31099 [0.06170204]
index: 31100 [0.07855654]
index: 31101 [0.16557983]
index: 31102 [0.08484107]
index: 31103 [0.10375401]
index: 31104 [0.09184906]
index: 31105 [0.01771265]
index: 31106 [0.25017354]
index: 31107 [0.5433979]
index: 31108 [0.6643985]
index: 31109 [0.17557973]
index: 31110 [0.36381194]
index: 31111 [0.04196259]
index: 31112 [0.07598373]
index: 31113 [0.366983

In [3]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_bank_marketing
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_bank_marketing.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            }

model_path = "models/gated_models/bank_a_gated_4_0.3_0.2_p-0.15_p0.3.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('dense_132', 'scale_layer_5'), ('dense_133', 'scale_layer_6'), ('dense_134', 'scale_layer_7'), ('dense_135', 'scale_layer_8'), ('dense_136', 'layer5')]

from preprocessing import pre_bank_marketing
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_bank_marketing.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_bank_marketing.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_bank_marketing.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                510       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)                15  

In [8]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_census_income
import tensorflow.keras.backend as K
import argparse
from scalelayer import  ScaleLayer

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred


def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

model_map = {
    'a': "models/diff_adult_a_gated_4_diff.h5",
    'r': "models/diff_adult_r_gated_4_diff.h5",
    'g': "models/diff_adult_g_gated_4_diff.h5",
    'a&r': "models/diff_adult_a&r_gated_4_diff.h5",
    'a&g': "models/diff_adult_a&g_gated_4_diff.h5",
    'r&g': "models/diff_adult_r&g_gated_4_diff.h5",
}

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'layer5')]

from preprocessing import pre_census_income
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]
        
        model_path = model_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        model.summary()

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_census_income.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_census_income.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_census_income.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))
                                 

layer 0
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)            

attr: a&g 0.096
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)    

attr: a&r 0.028
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)    

attr: g 0.001
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r 0.001
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: a 0.003
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 12)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r&g 0.053


In [8]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_bank_marketing
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_bank_marketing.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            }

model_path = "models/gated_models_diff/diff_bank_a_gated_4_diff.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('dense_132', 'scale_layer_5'), ('dense_133', 'scale_layer_6'), ('dense_134', 'scale_layer_7'), ('dense_135', 'scale_layer_8'), ('dense_136', 'layer5')]

from preprocessing import pre_bank_marketing
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_bank_marketing.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_bank_marketing.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_bank_marketing.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 30)                510       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 20)                20        
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)                15  

In [4]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_lsac
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(50, activation="relu", input_shape=pre_lsac.X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(int(constraint[i][0]), int(constraint[i][1]+1)))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 
            'r': [10],
            'g': [9],
            'g&r': [10, 9],
            }

models_map = {
            'r': 'models/gated_models/lsac_r_gated_4_0.3_0.2_p-0.9_p0.05.h5',
            'g': 'models/gated_models/lsac_g_gated_4_0.3_0.2_p-0.85_p0.2.h5',
            'g&r': 'models/gated_models/lsac_g&r_gated_4_0.3_0.2_p-0.55_p0.45.h5',
}



# similar_X = [dis_data, new_data]

layer_map = [('layer5', 'scale_layer_5'), ('layer5', 'scale_layer_6'), ('layer5', 'scale_layer_7'), ('layer5', 'scale_layer_8'), ('layer5', 'layer5')]

from preprocessing import pre_lsac
from tensorflow.keras.models import Model

for layer_index in range(0, 5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)
        model_path = models_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        model.summary()

        dis_data = pre_lsac.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_lsac.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_lsac.X_train)
        
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

layer 0
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)            

attr: g 0.089
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r 0.072
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: g&r 0.073


In [9]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_lsac
import tensorflow.keras.backend as K
import argparse
from scalelayer import  ScaleLayer

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(50, activation="relu", input_shape=pre_lsac.X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(int(constraint[i][0]), int(constraint[i][1]+1)))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 
            'r': [10],
            'g': [9],
            'g&r': [10, 9],
            }

models_map = {
            'r': 'models/gated_models_diff/diff_lsac_r_gated_4_diff.h5',
            'g': 'models/gated_models_diff/diff_lsac_g_gated_4_diff.h5',
            'g&r': 'models/gated_models_diff/diff_lsac_g&r_gated_4_diff.h5',
}



# similar_X = [dis_data, new_data]

layer_map = [('layer5', 'scale_layer_5'), ('layer5', 'scale_layer_6'), ('layer5', 'scale_layer_7'), ('layer5', 'scale_layer_8'), ('layer5', 'layer5')]

from preprocessing import pre_lsac
from tensorflow.keras.models import Model

for layer_index in range(0, 5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)
        model_path = models_map[attr]
        # adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
        model = tf.keras.models.load_model(model_path, custom_objects={'ScaleLayer': ScaleLayer})
        model.summary()

        dis_data = pre_lsac.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_lsac.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][1]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_lsac.X_train)

        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()
        
        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum()

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

layer 0
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)            

attr: g 0.121
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: r 0.063
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
scale_layer_5 (ScaleLayer)   (None, 50)                50        
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
scale_layer_6 (ScaleLayer)   (None, 30)                30        
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
scale_layer_7 (ScaleLayer)   (None, 15)      

attr: g&r 0.024
